In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [23]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBSQLiteConnection, CreateDealsTableIfNotExist
from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson, FetchEventsRecordAll
from DBUtility.DBUtility import InsertDealDoneRecord, FetchDealsDoneRecordLastThree
from pandas import DataFrame
import datetime
CoreConfig.initialize_core_config()

DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
CreateDealsTableIfNotExist.execute()


Initialize NAB Done Deals table            failed. Please Check your connection
table nab_deals_events_board already exists


In [24]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
from StylingUtility.StylingUtility import CustomStyling

In [25]:
title = widgets.HTML("<center><H3>DEALS Done</H3><center>")
df_out = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})
with df_out:
    df = DataFrame(FetchDealsDoneRecordLastThree().execute(), columns=['Company_name', 'Deal_Done_Date', 'Invested_in', 'AUD_Invested'])
    display(df)

date = widgets.DatePicker(
        value = datetime.datetime.now().date(),
        description='Pick a Date',
        layout=CustomStyling.box_layout,
        style=CustomStyling.style,
        disable=False)

company_name=widgets.Text(
    value='',
    placeholder='Type company\'s name',
    description='Company:',
    layout=CustomStyling.box_layout,
    style=CustomStyling.style,
    disable=False)

interested_in = widgets.Text(value='',
                             placeholder='Invested in',
                             description='Invested in:',
                             layout=CustomStyling.box_layout,
                             style=CustomStyling.style,
                             disable=False)

invested = widgets.BoundedFloatText(value=0.0,
                                    min=0, max=1000000,
                                    description='$ Invested',
                                    disabled=False)

button = widgets.Button(description='Submit the form',
                        disable=False,
                        button_style ='', # success
                        tooltip='Click to submit',
                        icon='check')

def on_submit_form_button_clicked(b):
    
    InsertDealDoneRecord(date.value, company_name.value, interested_in.value, invested.value).execute()
    #df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    
    df = DataFrame(FetchDealsDoneRecordLastThree().execute(), columns=['Company_name', 'Deal_Done_Date', 'Invested_in', 'AUD_Invested'])
    df_out.clear_output()
    with df_out:
        display(df)
button.on_click(on_submit_form_button_clicked)

In [26]:
def form(date, company_name, interested_in, invested):
    #display(events, meeting, date, name, notes, country, audience, website, asset, person)
    return date, company_name, interested_in, invested

In [27]:
ui = widgets.VBox(children=[df_out,title,
                            date, company_name, interested_in, invested, button], layout=CustomStyling.items_layout)

In [28]:
out = interactive_output(form, {'date': date,
                                'company_name': company_name,
                                'interested_in' : interested_in,
                                'invested' : invested
                               })
display(ui, out)


Output()

In [ ]:
person.options = {}